In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd

import cabi.etl.extract as extract
import cabi.cleaning as clean
import cabi.etl.transform as t

from geoalchemy2 import Geometry, WKTElement
from sqlalchemy import create_engine

## This Notebook Will be for Cleaning/Transforming Data from Raw CSV files and Loading into a PostGIS database

In [2]:
# List of filenames
a, b, c = extract.build_paths(csv_url='csv')

csv_paths = extract.compile_paths(a,b,c)

raw_dfs = extract.import_csvs(csv_paths)



## The Columns Are Not All The Same

- Observe recent difference in col naming convention for the set

In [17]:
col_names = [df.columns for df in raw_dfs]
col_names

[Index(['Duration', 'Start date', 'End date', 'Start station number',
        'Start station', 'End station number', 'End station', 'Bike number',
        'Member type'],
       dtype='object'),
 Index(['Duration', 'Start date', 'End date', 'Start station number',
        'Start station', 'End station number', 'End station', 'Bike number',
        'Member type'],
       dtype='object'),
 Index(['Duration', 'Start date', 'End date', 'Start station number',
        'Start station', 'End station number', 'End station', 'Bike number',
        'Member type'],
       dtype='object'),
 Index(['Duration', 'Start date', 'End date', 'Start station number',
        'Start station', 'End station number', 'End station', 'Bike number',
        'Member type'],
       dtype='object'),
 Index(['Duration', 'Start date', 'End date', 'Start station number',
        'Start station', 'End station number', 'End station', 'Bike number',
        'Member type'],
       dtype='object'),
 Index(['Duration', 'Star

## But the Information Is Roughly The Same
- Notably different items are:
    - Rideable Type (All of the old sets are presumed to be Docked Bikes)
    - Missing Lat/Lng (need an imputation strategy)
    - Different Station ID/Numbering Convention (not a huge deal)
    - No Unique Ride ID (we'll want to add this if using this Data)

In [18]:
raw_dfs[0]

,Duration,Start date,End date,Start station number,Start station,End station number,End station,Bike number,Member type
0,552,2018-01-01 00:05:06,2018-01-01 00:14:18,31104,Adams Mill & Columbia Rd NW,31400,Georgia & New Hampshire Ave NW,W00886,Member
1,1282,2018-01-01 00:14:30,2018-01-01 00:35:53,31321,15th St & Constitution Ave NW,31321,15th St & Constitution Ave NW,W01435,Casual
2,1265,2018-01-01 00:14:53,2018-01-01 00:35:58,31321,15th St & Constitution Ave NW,31321,15th St & Constitution Ave NW,W21242,Casual
3,578,2018-01-01 00:15:31,2018-01-01 00:25:09,31406,14th & Upshur St NW,31103,16th & Harvard St NW,W21322,Casual
4,372,2018-01-01 00:18:02,2018-01-01 00:24:15,31618,4th & East Capitol St NE,31619,Lincoln Park / 13th & East Capitol St NE,W00119,Member
...,...,...,...,...,...,...,...,...,...
168585,1377,2018-01-31 23:50:17,2018-02-01 00:13:14,31200,Massachusetts Ave & Dupont Circle NW,31265,5th St & Massachusetts Ave NW,W21414,Casual
168586,761,2018-01-31 23:52:43,2018-02-01 00:05:24,31603,1st & M St NE,31628,1st & K St SE,W23213,Member
168587,738,2018-01-31 23:54:27,2018-02-01 00:06:45,31107,Lamont & Mt Pleasant NW,31114,18th St & Wyoming Ave NW,W21373,Member
168588,247,2018-01-31 23:56:31,2018-02-01 00:00:38,31623,Columbus Circle / Union Station,31612,D St & Maryland Ave NE,W20936,Member


## For Now, Let's Just Use the Recent Data

In [19]:
april_to_july_list = raw_dfs[-4:]
april_to_july_list[1] = april_to_july_list[1].drop('is_equity', axis=1)

raw_apr_to_jul_df = pd.concat(april_to_july_list)
raw_apr_to_jul_df

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,495C10604A58ACA9,docked_bike,2020-04-27 10:33:08,2020-04-27 10:53:26,4th & M St SW,21.0,New York Ave & 15th St NW,124.0,38.876700,-77.017800,38.899000,-77.033400,member
1,08A5138C88D182A5,docked_bike,2020-04-27 11:20:07,2020-04-27 11:42:55,17th & K St NW,100.0,4th & M St SW,21.0,38.902800,-77.038600,38.876700,-77.017800,member
2,505E308BDCA9C792,docked_bike,2020-04-27 09:45:45,2020-04-27 09:54:32,1st & M St SE,578.0,4th & M St SW,21.0,38.876900,-77.006100,38.876700,-77.017800,member
3,D3436FC6CCF04110,docked_bike,2020-04-14 13:16:54,2020-04-14 13:28:59,11th & Kenyon St NW,15.0,Thomas Circle,178.0,38.929500,-77.027800,38.905900,-77.032500,casual
4,37B8457C89257BD5,docked_bike,2020-04-19 20:42:25,2020-04-19 20:51:12,13th & H St NE,97.0,Eastern Market / 7th & North Carolina Ave SE,96.0,38.900400,-76.988300,38.887000,-76.996800,member
...,...,...,...,...,...,...,...,...,...,...,...,...,...
228297,CD03F80ACB9AF2F4,docked_bike,2020-07-31 15:15:45,2020-07-31 16:25:22,5th & Kennedy St NW,86.0,Mount Vernon Ave & Four Mile Run Park,455.0,38.956555,-77.019814,38.843422,-77.064016,member
228298,A0060CB4B9B17F32,docked_bike,2020-07-20 19:35:00,2020-07-20 19:56:18,TJ Cmty Ctr / 2nd St & S Old Glebe Rd,361.0,N Lynn St & Fairfax Dr,491.0,38.869418,-77.095596,38.893860,-77.070862,casual
228299,E7BC1FE6BF9571F4,docked_bike,2020-07-01 16:56:24,2020-07-01 19:44:04,22nd & P ST NW,437.0,Adams Mill & Columbia Rd NW,17.0,38.909394,-77.048728,38.922925,-77.042581,casual
228300,B73B5EB431B2BA1E,docked_bike,2020-07-29 21:00:38,2020-07-29 21:35:10,Maine Ave & 7th St SW,95.0,Maine Ave & 7th St SW,95.0,38.878693,-77.023054,38.878694,-77.023054,casual


### About 35s to run clean_frame or to_geo on 100000 rows

In [22]:
%%time
clean.full_transform(test_df)

CPU times: user 1min 33s, sys: 462 ms, total: 1min 34s
Wall time: 1min 34s


,ride_id,start_end,duration_seconds,rideable_type,member_casual,ANC,station_name,coord
time,,,,,,,,
2020-04-01 00:25:48,4B5CEB51ACB3FFD5,start,131.0,docked_bike,member,Outside,Eads St & 15th St S,POINT (-77.05320 38.85900)
2020-04-01 00:27:59,4B5CEB51ACB3FFD5,end,131.0,docked_bike,member,Outside,Eads St & 12th St S,POINT (-77.05430 38.86280)
2020-04-01 01:36:09,1AF6FB400919CFB9,start,188.0,docked_bike,member,2F,14th & Rhode Island Ave NW,POINT (-77.03230 38.90860)
2020-04-01 01:39:17,1AF6FB400919CFB9,end,188.0,docked_bike,member,2F,12th & L St NW,POINT (-77.02840 38.90380)
2020-04-01 01:47:36,756AD339589371A3,start,699.0,docked_bike,member,1A,14th & Irving St NW,POINT (-77.03220 38.92860)
...,...,...,...,...,...,...,...,...
2020-05-31 21:58:03,FC890CE8E9A1062E,end,1287.0,docked_bike,casual,Outside,Potomac Ave & Swann Ave,POINT (-77.04780 38.82950)
2020-05-31 22:01:51,101B61FD21E30E9A,end,1266.0,docked_bike,casual,Outside,Barton St & 10th St N,POINT (-77.08780 38.88500)
2020-05-31 22:20:14,908ED2FD2CAC3B5B,end,11544.0,docked_bike,casual,Outside,Lynn St & 19th St North,POINT (-77.07100 38.89730)


In [11]:
%%time
cleaned_recent = clean.clean_frame(raw_apr_to_jul_df)
cleaned_recent

CPU times: user 781 ms, sys: 111 ms, total: 892 ms
Wall time: 898 ms


,ride_id,rideable_type,started_at,ended_at,start_station_name,end_station_name,start_lat,start_lng,end_lat,end_lng,member_casual,duration_seconds
0,495C10604A58ACA9,docked_bike,2020-04-27 10:33:08,2020-04-27 10:53:26,4th & M St SW,New York Ave & 15th St NW,38.876700,-77.017800,38.899000,-77.033400,member,1218.0
1,08A5138C88D182A5,docked_bike,2020-04-27 11:20:07,2020-04-27 11:42:55,17th & K St NW,4th & M St SW,38.902800,-77.038600,38.876700,-77.017800,member,1368.0
2,505E308BDCA9C792,docked_bike,2020-04-27 09:45:45,2020-04-27 09:54:32,1st & M St SE,4th & M St SW,38.876900,-77.006100,38.876700,-77.017800,member,527.0
3,D3436FC6CCF04110,docked_bike,2020-04-14 13:16:54,2020-04-14 13:28:59,11th & Kenyon St NW,Thomas Circle,38.929500,-77.027800,38.905900,-77.032500,casual,725.0
4,37B8457C89257BD5,docked_bike,2020-04-19 20:42:25,2020-04-19 20:51:12,13th & H St NE,Eastern Market / 7th & North Carolina Ave SE,38.900400,-76.988300,38.887000,-76.996800,member,527.0
...,...,...,...,...,...,...,...,...,...,...,...,...
228297,CD03F80ACB9AF2F4,docked_bike,2020-07-31 15:15:45,2020-07-31 16:25:22,5th & Kennedy St NW,Mount Vernon Ave & Four Mile Run Park,38.956555,-77.019814,38.843422,-77.064016,member,4177.0
228298,A0060CB4B9B17F32,docked_bike,2020-07-20 19:35:00,2020-07-20 19:56:18,TJ Cmty Ctr / 2nd St & S Old Glebe Rd,N Lynn St & Fairfax Dr,38.869418,-77.095596,38.893860,-77.070862,casual,1278.0
228299,E7BC1FE6BF9571F4,docked_bike,2020-07-01 16:56:24,2020-07-01 19:44:04,22nd & P ST NW,Adams Mill & Columbia Rd NW,38.909394,-77.048728,38.922925,-77.042581,casual,10060.0
228300,B73B5EB431B2BA1E,docked_bike,2020-07-29 21:00:38,2020-07-29 21:35:10,Maine Ave & 7th St SW,Maine Ave & 7th St SW,38.878693,-77.023054,38.878694,-77.023054,casual,2072.0


In [12]:
geo = clean.to_geo(cleaned_recent)

In [15]:
geo = clean.rename_columns(geo)

In [20]:
long_geo = clean.full_transform(raw_apr_to_jul_df)
long_geo

,ride_id,start_end,duration_seconds,rideable_type,member_casual,ANC,station_name,coord
time,,,,,,,,
2020-04-01 00:25:48,4B5CEB51ACB3FFD5,start,131.0,docked_bike,member,Outside,Eads St & 15th St S,POINT (-77.05320 38.85900)
2020-04-01 00:27:59,4B5CEB51ACB3FFD5,end,131.0,docked_bike,member,Outside,Eads St & 12th St S,POINT (-77.05430 38.86280)
2020-04-01 01:36:09,1AF6FB400919CFB9,start,188.0,docked_bike,member,2F,14th & Rhode Island Ave NW,POINT (-77.03230 38.90860)
2020-04-01 01:39:17,1AF6FB400919CFB9,end,188.0,docked_bike,member,2F,12th & L St NW,POINT (-77.02840 38.90380)
2020-04-01 01:47:36,756AD339589371A3,start,699.0,docked_bike,member,1A,14th & Irving St NW,POINT (-77.03220 38.92860)
...,...,...,...,...,...,...,...,...
2020-08-01 01:52:27,0C79916883F5FFA0,end,9408.0,docked_bike,casual,2C,10th St & Constitution Ave NW,POINT (-77.02601 38.89303)
2020-08-01 01:56:45,AC57E5DE2CFBAA8C,end,7272.0,docked_bike,casual,2E,34th & Water St NW,POINT (-77.06753 38.90373)
2020-08-01 02:50:22,31D02200D90B2080,end,14246.0,docked_bike,member,6D,10th St & L'Enfant Plaza SW,POINT (-77.02601 38.88432)


time
2020-04-01 00:25:48      1.0
2020-04-01 00:27:59      2.0
2020-04-01 01:36:09      3.0
2020-04-01 01:39:17      4.0
2020-04-01 01:47:36      5.0
                       ...  
2020-08-01 01:52:27    589.0
2020-08-01 01:56:45    551.0
2020-08-01 02:50:22    170.0
2020-08-01 02:50:28    170.0
2020-08-01 08:13:39      1.0
Name: duration_seconds, Length: 1319554, dtype: float64

In [27]:
# Creating SQLAlchemy's engine to use
engine = pass

In [26]:


geodataframe = long_geo.copy()
#... [do something with the geodataframe]

geodataframe['geom_coord'] = geodataframe['coord'].apply(lambda x: WKTElement(x.wkt, srid=4326))


                                                               
#drop the geometry column as it is now duplicative
geodataframe.drop('coord', axis=1, inplace=True)

# Use 'dtype' to specify column's type
# For the geom column, we will use GeoAlchemy's type 'Geometry'
geodataframe.to_sql(
    'trips_long',
    engine,
    if_exists='append',
    index=True,
    index_label='time',
    chunksize=10000,
    dtype={
        'geom_coord': Geometry(
            geometry_type='POINT',
            srid=4326)
    })

In [24]:
plus_minus = t.to_plus_minus(long_geo)

In [28]:
plus_minus['1A'].rolling('3H').sum()

time
2020-04-01 00:25:48     0.0
2020-04-01 00:27:59     0.0
2020-04-01 01:36:09     0.0
2020-04-01 01:39:17     0.0
2020-04-01 01:47:36    -1.0
                       ... 
2020-08-01 01:52:27     9.0
2020-08-01 01:56:45    10.0
2020-08-01 02:50:22     4.0
2020-08-01 02:50:28     4.0
2020-08-01 08:13:39     0.0
Name: 1A, Length: 1319554, dtype: float64

In [29]:
plus_minus.to_sql(
    'plus_minus',
    engine,
    if_exists='append',
    index=True,
    index_label='time'
)


In [30]:
# Ensure connections are closed

engine.dispose()

# IT WORKS!

## There are no lat/lng coords in the Historical DataFrame: Can we Impute Coords from Recent Station Info?

## Second Note on Approach -- Should we be modeling checkin and checkouts instead of gain/loss?

In [ ]:

def prop_top(col):
    """Return the proportion of the total observations that the mode observation makes up for a given pandas Series"""
    rounded = round(col, 4)
    count_top = rounded.value_counts().values[0]
    total = len(rounded)
    prop = count_top / total
    return prop

def count_obs(col, name):
    """Return number of instances of name in col"""
    
    count = len(col[col == name])
    return count

stations = list(raw_apr_to_jul_df.start_station_name.value_counts().index)
stations.remove('Motivate Tech Office')
prop_lats = {name: prop_top(raw_apr_to_jul_df[raw_apr_to_jul_df.start_station_name == name].start_lat)\
             for name in stations}

prop_lngs = {name: prop_top(raw_apr_to_jul_df[raw_apr_to_jul_df.start_station_name == name].start_lng)\
             for name in stations}

prop_lats = {k: v for k, v in sorted(prop_lats.items(), key=lambda item: item[1])}
prop_lngs = {k: v for k, v in sorted(prop_lngs.items(), key=lambda item: item[1])}


# station_counts = {station: count_obs(raw_apr_to_jul_df['start_station_name'], station) for station in stations}
# station_counts

## In General It Seems very reasonable to impute start_lat/lng for each station from top value_count, there are 16 stations for which either lat or lng impute strategy is less than 90% accurate
- Saving these in a list, visualize to look for pattern or outliers

In [ ]:
inaccurate_keys = []
for key in prop_lats.keys():
    if (prop_lats[key] < 0.9) | (prop_lngs[key] < 0.9):
        print(f'{key}: {prop_lats[key]}, {prop_lngs[key]}')
        inaccurate_keys.append(key)

inaccurate_keys
    

In [ ]:
# raw_july = pd.read_csv(csv_paths[-1])

# raw_dockless = raw_july[raw_july.rideable_type == 'electric_bike']
# pd.to_pickle(raw_dockless, '../data/wip/raw_dockless.pkl')
# pd.to_pickle(gdf, '../data/wip/gdf_anc.pkl')

gdf = get_data.anc_gdf()

def plot_trips(df, axe):
    gdf.plot(figsize=(10, 10), alpha=0.5, edgecolor='k', ax=axe)
    sns.scatterplot(x='start_lng', y='start_lat', data=df, ax=axe)

## No Readily Discernable Pattern Geographically to the no end lat/lng trips

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))

plot_trips(raw_apr_to_jul_df[raw_apr_to_jul_df.end_lat.isna()], axe=ax)

In [ ]:
fig, axes = plt.subplots(8,2, figsize=(10, 30))
axes = axes.ravel()

for i, key in enumerate(inaccurate_keys):
    df = raw_apr_to_jul_df.loc[
        raw_apr_to_jul_df['start_station_name'] == key, ['start_lng', 'start_lat']]
    sns.scatterplot(x='start_lng', y='start_lat', data=df, ax=axes[i])
    axes[i].set_title(key)

fig.tight_layout()
plt.show()

In [ ]:
pd.to_pickle(raw_apr_to_jul_df, '../data/wip/raw_apr_to_jul_df.pkl')

In [ ]:
df = raw_apr_to_jul_df.loc[
    raw_apr_to_jul_df['start_station_name'] == key, ['start_lng', 'start_lat']

## Running Notes
- Feature engineer churn metric for each station? Then predict continuous outcome?